# Bias and limitations

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and p

In [2]:
from transformers import pipeline

unmasker = pipeline("fill-mask", model="bert-base-uncased")
result = unmasker("This man works as a [MASK].")
print([r["token_str"] for r in result])

result = unmasker("This woman works as a [MASK].")
print([r["token_str"] for r in result])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


['carpenter', 'lawyer', 'farmer', 'businessman', 'doctor']
['nurse', 'maid', 'teacher', 'waitress', 'prostitute']


### **预训练模型的偏见与局限性**  

尽管预训练模型（如BERT、GPT等）是强大的工具，但在生产环境中使用时，必须意识到它们存在**固有的偏见和局限性**。这些偏见主要源于训练数据的来源——互联网上的海量文本往往包含社会偏见、歧视性内容和不平等现象。  

---

### **1. 偏见的典型案例**  
#### **示例：BERT 的性别偏见**  
使用 `fill-mask` 任务测试 **BERT-base-uncased**：  
```python
from transformers import pipeline

unmasker = pipeline("fill-mask", model="bert-base-uncased")

# 测试男性职业关联
print(unmasker("This man works as a [MASK]."))
# 输出: ['lawyer', 'carpenter', 'doctor', 'waiter', 'mechanic']

# 测试女性职业关联
print(unmasker("This woman works as a [MASK]."))
# 输出: ['nurse', 'waitress', 'teacher', 'maid', 'prostitute']
```
**问题发现**：  
- 模型对男性和女性的职业预测存在明显**性别刻板印象**：  
  - 男性关联到“律师、医生”等高地位职业。  
  - 女性关联到“护士、女佣”甚至“性工作者”等带有偏见的词汇。  
- 即使 BERT 的训练数据（Wikipedia 和 BookCorpus）相对“中性”，仍无法避免此类偏见。  

---

### **2. 偏见的根源**  
1. **训练数据的社会偏见**  
   - 互联网文本本身反映现实社会中的性别、种族、文化等不平等现象。  
   - 例如：历史文本中“医生”多为男性，“护士”多为女性，导致模型学习到这种统计规律。  

2. **模型放大偏见**  
   - 语言模型通过概率预测下一个词，可能强化数据中的偏见模式（如“女人更适合当家庭主妇”）。  

3. **微调无法完全消除偏见**  
   - 即使在小规模标注数据上微调，底层模型的偏见仍可能通过迁移学习保留。  

---

### **3. 其他常见偏见类型**  
| **偏见类型** | **示例**                                                                 |
|--------------|--------------------------------------------------------------------------|
| **种族偏见** | 将某些种族与犯罪、贫困等负面词汇关联。                                   |
| **宗教偏见** | 对特定宗教群体生成歧视性描述。                                           |
| **文化偏见** | 以西方为中心的世界观（如默认“圣诞节”是全球性节日）。                     |
| **能力偏见** | 将残疾人士与“脆弱”“需要帮助”等词汇强关联。                               |

---

### **4. 如何应对偏见？**  
#### **（1）数据层面**  
- **清洗训练数据**：过滤仇恨言论、歧视性内容（但难以完全覆盖）。  
- **平衡数据分布**：主动增加少数群体的代表性（如女性科学家文本）。  

#### **（2）模型层面**  
- **去偏算法**：  
  - **对抗训练**：通过对抗网络抑制偏见特征的表达。  
  - **重新加权**：降低偏见相关词的概率（如手动调整“prostitute”的得分）。  
- **可控生成**：使用提示工程（Prompting）引导模型输出中性内容（如明确添加“请避免性别偏见”）。  

#### **（3）应用层面**  
- **后处理过滤**：对生成内容进行敏感词检测和替换。  
- **人工审核**：关键场景（如客服、招聘）需人工复核模型输出。  

---

### **5. 开发者责任**  
- **透明性**：告知用户模型可能存在偏见。  
- **监控与迭代**：持续评估模型在真实场景中的表现，更新去偏策略。  
- **伦理审查**：在医疗、法律等高风险领域，建立伦理评估流程。  

---

### **6. 实例：减少偏见的实践**  
```python
# 示例：通过提示工程减轻性别偏见
from transformers import pipeline

generator = pipeline("text-generation", model="gpt2")

# 不安全的提示
output = generator("The nurse should be...", max_length=30)
print(output[0]["generated_text"])  # 可能输出刻板印象内容

# 改进后的提示
output = generator("The nurse, regardless of gender, should be...", max_length=30)
print(output[0]["generated_text"])  # 更中性的输出
```

---

### **关键结论**  
- **所有预训练模型都存在偏见**，这是数据驱动的本质决定的。  
- **偏见无法彻底消除**，但可通过技术和管理手段缓解。  
- **开发者必须主动应对**，避免模型在真实世界中加剧不平等。  

> **提示**：Hugging Face 提供了部分去偏工具（如`DebiasBERT`），但需结合业务需求定制化处理。